In [ ]:
import subprocess
from subprocess import Popen
import json
import os

In [ ]:
expe_name='4-23'   #'4-23'

#CFs=[3000, 4000, 5000, 6000, 8000]

CFs=[2200, 3000, 4000, 5000, 6000, 8000]
E0_distributed=True
Q10_distributed=True

I0_distributed=False
results_folder0=f'./results/fit{expe_name}-distrib/' #if I0_distributed, loads wb cdf params from other folder

write_results=True

filter_model='gammatone_4'

results_name='test-0609'  #if not blank, will save all the results in a folder with results_name (also loads param from this folder, like optim params)
results_folder=None
if results_name != '':
    results_folder=f'./results/fit{expe_name}-{results_name}/'
    

load_json_optim_params=True #if True, will load optim params from optim_params.json
load_json_init_params=True #if True, will load ./init_params/{expe_name}/{CF}_init_params.json if exists


In [ ]:
#writes a temporary json file to share params with the RBF net main process


json_dic={'expe_name':expe_name, 'E0_distributed':E0_distributed, 'Q10_disbtributed':Q10_distributed,
         'I0_distributed':I0_distributed, 'n_workers':str(len(CFs)+1), 'write_results':write_results, 'CFs':CFs,
         "results_folder0": results_folder0, 
          'load_json_optim_params': load_json_optim_params}
if results_folder is not None:
    json_dic['results_name']=results_name
    json_dic['results_folder']=results_folder
with open(f'distrib_params.json', 'w') as outfile:
    json.dump(json_dic, outfile, indent=4)
    
#if results_folder is not None: #writes in results folder as well
#    os.makedirs(results_folder)
#    with open(f'distrib_params.json', 'w') as outfile:
#        json.dump(json_dic, outfile, indent=4)

In [ ]:
use_screen=True
list_args0=['papermill', '-p', 'E0_distributed',  str(E0_distributed), 
       '-p', 'Q10_distributed', str(Q10_distributed), 
            '-p', 'I0_distributed',  str(I0_distributed),
            '-p', 'n_workers',
       str(len(CFs)+1), '-p', 'write_results', str(write_results),
            '-p', 'expe_name', expe_name, '-p', 'filter_model', filter_model,
            '-p', 'load_json_init_params', load_json_init_params,
            
         '-p', 'load_json_optim_params', load_json_optim_params]

if I0_distributed:
    list_args0+=['-p', 'results_folder0', results_folder0]
    
if results_folder is not None:
    list_args0+=['-p', 'results_name', results_name, '-p', 'results_folder', results_folder]
    


processes=[]
for i, CF in enumerate(CFs):
    list_args=list_args0+ ['-p', 'rank', str(i+1), '-p',  'CF', str(CF), 'Fit\ data.ipynb', f'fitdata{CF}.ipynb']
    st_cmd=' '.join([str(arg) for arg in list_args])
    if use_screen:
        #st_cmd=f'screen -X -S {CF} ' + st_cmd 
        st_cmd=f"screen -S {CF} -dm {st_cmd}"
    print(st_cmd)
    #processes.append(Popen(st_cmd, shell=True))

move all param files in results folder (for tracking params after changes)

In [ ]:
if write_results:
    if results_folder is None:
        if Q10_distributed:
            if I0_distributed:

                results_folder2=f'./results/fit{expe_name}-distrib/I0_distrib/'
            else:
                results_folder2=f'./results/fit{expe_name}-distrib/'

        else:
            results_folder2=f'./results/fit{expe_name}/'
    else:
        results_folder2=results_folder
        
    if not os.path.exists(results_folder2):
        os.makedirs(results_folder2)
    
    if os.path.exists(f'optim_params_{expe_name}.json'):
        optim_params_filename=f'optim_params_{expe_name}.json'
    else:
        optim_params_filename='optim_params.json'
        
    list_filenames=['distrib_params.json', 'E0_params.json', optim_params_filename]
    
    if Q10_distributed:
        list_filenames.append('RBF_params.json')
    if I0_distributed:
        list_filenames.append('RBF_I0_params.json')
    for filename in list_filenames:
        subprocess.run(['cp', filename, f'{results_folder2}/{filename}'])
        
    if load_json_init_params:
        subprocess.run(['cp', '-R', f'./init_params/{expe_name}/', f'{results_folder2}/init_params/'  ])
    print(f'param files copied to {results_folder2}')

**Don't forget to copy the jupyter notebooks in results folder to keep track of the figures for each run!**